In [2]:
import pandas as pd

##Read the sample file from local storage
sample_data=pd.read_csv('../sample_comment.csv')
sample_data.head()

,content_id,channel,comment_id,content,comment_time,ref_datetime
0,VMfwN-XHiJs,youtube,UgybSDK1DreyYAP3il14AaABAg,可能他喜歡別人😭,2021-05-27 08:40:26+00,2022-05-05 22:41:48.966+00
1,VMfwN-XHiJs,youtube,Ugywq8IS5gNcfn-G7GV4AaABAg,作者，那他是否會做出一些讓你以為他喜歡你的行為，但是其實他不喜歡你只是你自己覺得而已？,2021-05-27 08:40:26+00,2022-05-05 22:41:48.966+00
2,VMfwN-XHiJs,youtube,Ugy2llUeMTATQRck0-t4AaABAg,恐女表示OTZ,2021-05-27 08:40:26+00,2022-05-05 22:41:48.966+00
3,VMfwN-XHiJs,youtube,UgyYuZLLlYOaqLq39iF4AaABAg,一堆規則真的很快就冷掉,2021-05-27 08:40:26+00,2022-05-05 22:41:48.966+00
4,VMfwN-XHiJs,youtube,Ugwf76BqnP6zE-qnlu54AaABAg,可能他有女友。或已經結婚。 ^___^,2021-05-27 08:40:26+00,2022-05-05 22:41:48.966+00


In [8]:
##Convert all the "content_id" and "comment_id" to string for analysis later
comment_df = pd.DataFrame(sample_data, columns=["content_id", "channel", "comment_id", "content", "comment_time", "ref_datetime" ])
comment_df["content_id"], comment_df["comment_id"] = comment_df["content_id"].apply(str), comment_df["comment_id"].apply(str)

In [10]:
import re
import multiprocessing
from multiprocessing import Pool
import numpy as np
import emoji
NUM_PROC = multiprocessing.cpu_count()

In [11]:
# @calculate_time_cost
def label_with_emoji(comment_df, EMOJI_CSV):
    emoji_df = pd.read_csv(EMOJI_CSV, encoding="utf-8-sig")
    senti_re_df = emoji_df.groupby("sentiment").agg({"re_emoji": lambda cell: "|".join(cell)})
    pos_emoji_re = senti_re_df.loc["pos", "re_emoji"]
    neu_emoji_re = senti_re_df.loc["neu", "re_emoji"]
    neg_emoji_re = senti_re_df.loc["neg", "re_emoji"]

    for comment_df_index in comment_df[comment_df["pre_sentiment"].isna()].index:
        senti_sum = 0
        found_flag = False
        for emoji_re, coefficient in zip([pos_emoji_re, neu_emoji_re, neg_emoji_re], [1, 0, -1]):
            senti_sum += (num_found_emoji:=len(re.findall(emoji_re, comment_df.loc[comment_df_index, "content"]))) * coefficient
            if num_found_emoji: found_flag = True
        if found_flag:
            if senti_sum>0: senti = "pos"
            elif senti_sum<0: senti = "neg"
            else: senti = "neu"
            comment_df.loc[comment_df_index, ["pre_sentiment", "label_reason"]] = (senti, "emoji")
            comment_df.loc[comment_df_index, senti] = 1


    return comment_df

In [12]:
def string_preprocessing_with_content_col(comment_df):
    # print("string_preprocessing_with_content_col")
    REGEX_EMAIL = re.compile(r"[a-zA-Z0-9_\.-]+@[a-zA-Z0-9_-]+(?:\.[a-zA-Z0-9_-]+)+")
    REGEX_HTTP  = re.compile(r'https?:/?/?[a-zA-Z0-9.?%+/&#=:_-]*')                                         # http(s)://website +for search, %for Chinese, #for topic 
    REGEX_WWW   = re.compile(r"[^a-zA-Z0-9/]www\.[a-zA-Z0-9.?%+/&#=:_-]+|^www\.[a-zA-Z0-9.?%+/&#=:_-]+")    # www.website
    REGEX_COM   = re.compile(r"[a-zA-Z0-9?%+/&#=:_-]+\.com[a-zA-Z0-9.?%+/&#=:_-]+")                         # website.com.website
    REGEX_CODE = re.compile(r"&#+[0-9]+;")
    REGEX_2SPACE = re.compile(r"\n|\r|\t\f\v")
    REGEX_2COMMA = re.compile(r"、|，|；")
    REGEX_2Quot = re.compile(r"《|》|“|”|‘|’|「|」")
    REGEX_IGEMOJI =  re.compile(":_\w+:")
    
    PUNC = '~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——+-=“：’；、。，？》《{}'
    #for index in comment_df[comment_df["pre_sentiment"].isna()].index:
    for index in comment_df.index:
        text = str(comment_df.loc[index, "content"])
        text = text.replace('"',"")
        text = text.encode('unicode-escape').decode('unicode-escape')
        #print('*'*10,text)
        text = re.sub(REGEX_EMAIL, 'EMAIL', text)
        text = re.sub(REGEX_HTTP, 'HTTP', text)
        text = re.sub(REGEX_WWW, 'HTTP', text)
        text = re.sub(REGEX_COM, 'HTTP', text)
        text = re.sub(REGEX_CODE, "", text)                         # 遇到部分特殊字符无法识别
        #text = emojiswitch.demojize(text, delimiters = (" "," "), lang="en").replace("_", " ").replace("-", " ")
        text = emoji.replace_emoji(text, ' ')
        text = re.sub(REGEX_IGEMOJI, "", text)
        text = re.sub(r"(回复)?(//)?\s*@\S*?\s*( |:|：|$)", "", text)  # 去除正文中的@和回复/转发中的用户名
        text = re.sub(REGEX_2SPACE, " ", text)
        text = re.sub(REGEX_2COMMA,",", text)
        text = re.sub(REGEX_2Quot, "'", text)
        #text = text.replace("！","!").replace("？","?").replace("。",".")
        #text = re.sub(r'[0-9]+', "NUMBER", text)
        text = re.sub(r"\s+", " ", text)                                # 合并正文中过多的空格
        text = text.replace("。",".")
        text = re.sub(r"[%s]+" %PUNC, "",text)
        text = text.strip()                                             # 去掉字符串的左右两边空格
        comment_df.loc[index, "clean_content"] = text
    return comment_df


In [9]:
##Pre-processing of imported comments
def pre_processing_and_pre_predict(comment_df):
    if comment_df.empty:
        return comment_df
    # print("original shape", comment_df.shape)
    '0. initialization for sentiment'
    comment_df["pre_sentiment"] = np.nan
    comment_df["pre_sentiment"] = comment_df["pre_sentiment"].astype("object")

    comment_df['label_reason'] = None
    comment_df[['pos', 'neu', 'neg']] = 0,0,0
    # print('aaa comment_df["pre_sentiment"]', comment_df["pre_sentiment"])
    '''
    # sub_comment will be skipped 
    '1. Take off all subcomment from comment, then concat comments and subcomments as one.'
    comment_df = takeout_sub_comment_as_comment(comment_df)
    # print("turn_sub_comment_into_comment finished.")
    logging.info("turn_sub_comment_into_comment finished.")
    # print("adding subcomment", comment_df.shape)
    '''
    
    ''' **********************HOPE sn will be used later******************
    '2. Labelling sentiment directly according to the interact info.'
    comment_df = fun_sn_parsing(comment_df)
    # print("sentiment_count_with_interacts_col finished.")
    logging.info("sentiment_count_with_interacts_col finished.")
    comment_df.to_csv("2_sentiment_count_with_interacts_col.csv", encoding="utf-8-sig")
    '''

    '3. Labelling sentiment directly according to emoji.'
    comment_df = label_with_emoji(comment_df, EMOJI_CSV)
    comment_df.to_csv("test_csv/01_preprocess_label_with_emoji.csv", encoding="utf-8-sig")

    '4. pre-processing (data cleaning) for each comment string. '
    comment_df = string_preprocessing_with_content_col(comment_df)
    comment_df.to_csv("test_csv/02_preprocess_string_preprocessing_with_content_col.csv", encoding="utf-8-sig")
    # print("string_preprocessing_with_content_col finished.")
    #logging.info("string_preprocessing_with_content_col finished.")

    # '4. pre-processing for each comment string according to different languages. '
    # comment_df = string_preprocessing_with_content_col_for_multiple_lang(comment_df)
    # print("string_preprocessing_with_content_col finished.")
    # logging.info("string_preprocessing_with_content_col finished.")

    '5. label empty string as neutral. '
    comment_df = label_empty_string_as_neu(comment_df)
    #print("label_empty_string_as_pos finished.")
    #logging.info("label_empty_string_as_pos finished.")


    '6.1 label one letter as neutral. '
    comment_df = label_one_letter_as_neu(comment_df)
    print("6.1 label empty string as neutral finished.")
    #logging.info("6.1 label_one_letter_as_neu finished.")


    '6.2 label iterate string as positive. '
    comment_df = label_iterate_string_as_pos(comment_df)
    print("6.2 label_iterate_string_as_pos finished.")
    #logging.info("6.2 label_iterate_string_as_pos finished.")

    '6.2 label digital and special charactors as neutral. '
    comment_df = label_digital_special_charactors_as_neu(comment_df)
    print("6.2 label_digital_special_charactors_as_neu finished.")
    #logging.info("6.2 label_digital_special_charactors_as_neu finished.")

    '6.2 label unsupported unicode string as IGNORE. '
    comment_df = label_unsupported_unicode_as_IGNORE(comment_df)
    print("6.3 label_unsupported_unicode_as_neu finished.")
    #logging.info("6.3 label_unsupported_unicode_as_neu finished.")


    '''
    '6. Label all unlabelled comments that have the same content as labelled comments with the same label. '
    comment_df = copy_senti_from_labelled_comments_of_same_content(comment_df)
    # comment_df.to_csv("6. copy_senti_from_labelled_comments_of_same_content.csv", encoding="utf-8-sig")
    # print("copy_senti_from_labelled_comments_of_same_content finished.")
    logging.info("copy_senti_from_labelled_comments_of_same_content finished.")
    '''

    '7. Language Detection.'
    comment_df = language_detection_with_content_col(comment_df)
    comment_df.to_csv("test_csv/03_preprocess_language_detection_with_content_col.csv", encoding="utf-8-sig")
    # print("language_detection_with_content_col finished.")
    #logging.info("language_detection_with_content_col finished.")
    # comment_df = comment_df[:100]
    # print("comment_df", comment_df.shape[0])
    # a = language_detection_with_content_col_1(comment_df)
    # b = language_detection_with_content_col_2(comment_df)
    # c = language_detection_with_content_col_3(comment_df)
    # d = language_detection_with_content_col_4(comment_df)
    '''
    '8. top_k post comments per post'
    comment_df = Top_K_comments_per_post(comment_df)
    comment_df.to_csv("7_01_Top_K_comments_per_post.csv", encoding="utf-8-sig")
    print("7_01_Top_K_comments_per_post finished.")
    '''
    return comment_df


p = Pool( processes= NUM_PROC)
each_len = int(comment_df.shape[0]//num_proc)+1
# ret = p.map(pre_processing_and_pre_predict, [comment_df.loc[0:10, :] for i in range(num_proc)])
ret = p.map(pre_processing_and_pre_predict, [comment_df.iloc[i*each_len:(i+1)*each_len, :] for i in range(num_proc)])
p.close()
p.join()
# print(type(ret))
# print(len(ret))
# print(type(ret[0]))
# print(ret[0].shape)
# for index, item in enumerate(ret):
#     item.to_csv(f"ret_1214_{index}.csv", encoding="utf-8-sig")
ret = pd.concat(ret, axis=0).reset_index(drop=True)
